In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from unidecode import unidecode
import re
import unicodedata

In [2]:
def fill_district(truocsapnhap):
    parts = truocsapnhap.split(', ')
    current_district = None
    filled = []

    # Loop từ phải qua trái
    for part in reversed(parts):
        # Tìm district trong ngoặc
        m = re.search(r'\((.*?)\)', part)
        if m:
            current_district = m.group(1)
        # Nếu chưa có district thì bỏ qua
        if current_district:
            # Bỏ ngoặc cũ nếu có
            part = re.sub(r'\(.*?\)', '', part).strip()
            part = f"{part} ({current_district})"
        filled.append(part)

    # Đảo lại đúng thứ tự ban đầu
    filled = list(reversed(filled))
    return ', '.join(filled)

def extract_district(truocsapnhap):
    unit = truocsapnhap.split(' (')
    district = np.nan
    if len(unit) > 1:
        district = unit[1][:-1]
        if ' - Tỉnh ' in district:
            district = district.split(' - Tỉnh ')[0]
    return district

def extract_province(truocsapnhap):
    unit = truocsapnhap.split(' (')
    province = np.nan
    if len(unit) > 1:
        district = unit[1][:-1]
        if ' - Tỉnh ' in district:
            province = district.split(' - ')[1]
    return province

def extract_ward(truocsapnhap):
    unit = truocsapnhap.split(' (')
    ward = unit[0]
    return ward

def cap_first(text):
    if isinstance(text, str):
        split_text = text.split()
        split_text = [i[0].capitalize() + i[1:] if split_text.index(i)==0 else i for i in split_text]
        return ' '.join(split_text)
    return np.nan

def title_safely(text):
    if isinstance(text, str):
        split_text = text.split()
        split_text = [i[0].capitalize() + i[1:] for i in split_text]
        text = ' '.join(split_text)
        split_text = text.split("'")
        split_text = [i[0].capitalize() + i[1:] for i in split_text]
        return "'".join(split_text)
    return np.nan

def lower_safely(text):
    if isinstance(text, str):
        return text.lower()
    return np.nan

def normalize_quotes(text):
    if isinstance(text, str):
        return text.replace("’", "'").replace("‘", "'").replace("“", '"').replace("”", '"')
    return np.nan

def unidecode_pro(text):
    if isinstance(text, str):
        text = unidecode(text)
        text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)  # chỉ giữ a-z, A-Z, 0-9, thay phần còn lại bằng khoảng trắng
        text = re.sub(r'\s+', ' ', text)
        return text.strip().lower()
    return np.nan

def unicode_normalize(text):
    '''
    Chuyển Unicode tổ hợp sang Unicode dựng sẵn
    :param text:
    :return:
    '''
    if isinstance(text, str):
        text = unicodedata.normalize('NFC', text)
        text = normalize_quotes(text).strip()
        return text
    return np.nan

In [3]:
title_safely('ha nOi')

'Ha NOi'

In [4]:
title_safely("Thị Trấn Ea T'ling")

"Thị Trấn Ea T'Ling"

In [5]:
unicode_normalize("Cư K’nia")

"Cư K'nia"

# Chuẩn bị danh mục

In [6]:
df_danhmuc_district = pd.read_csv('data/danhmuchanhchinh.gso.gov.vn_district_2025-07-18.csv')

In [7]:
df_danhmuc_ward = pd.read_csv('data/danhmuchanhchinh.gso.gov.vn_2025-07-18.csv')

In [8]:
rename_cols = {
    'Mã': 'wardCode',
    'Mã QH': 'districtCode',
    'Mã TP': 'provinceCode',
    'Cấp': 'wardType',
    'Tên': 'ward',
    'Quận Huyện': 'district',
    'Tỉnh / Thành Phố': 'province'
}
df_danhmuc_ward = df_danhmuc_ward.rename(columns=rename_cols)[rename_cols.values()]

In [9]:
rename_cols = {
    'Mã': 'districtCode',
    'Mã TP': 'provinceCode',
    'Cấp': 'districtType',
    'Tên': 'district',
    'Tỉnh / Thành Phố': 'province'
}

df_danhmuc_district = df_danhmuc_district.rename(columns=rename_cols)[rename_cols.values()]

In [10]:
df_danhmuc_ward.shape

(10035, 7)

In [11]:
df_danhmuc_district.shape

(696, 5)

In [12]:
common = list(set(df_danhmuc_ward.columns.tolist()) & set(df_danhmuc_district.columns.tolist()))
df_danhmuc = pd.merge(df_danhmuc_district, df_danhmuc_ward, on=common, how='left')

In [13]:
df_danhmuc

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward
0,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,1.0,Phường,Phường Phúc Xá
1,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,4.0,Phường,Phường Trúc Bạch
2,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,6.0,Phường,Phường Vĩnh Phúc
3,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị
4,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,8.0,Phường,Phường Liễu Giai
...,...,...,...,...,...,...,...,...
10035,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32239.0,Xã,Xã Viên An Đông
10036,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32242.0,Xã,Xã Viên An
10037,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32244.0,Thị trấn,Thị trấn Rạch Gốc
10038,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân


In [14]:
df_danhmuc[df_danhmuc['ward'].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward
3582,318,31,Huyện,Huyện Bạch Long Vĩ,Thành phố Hải Phòng,NaN,NaN,NaN
5794,471,45,Huyện,Huyện Cồn Cỏ,Tỉnh Quảng Trị,NaN,NaN,NaN
5975,498,48,Huyện,Huyện Hoàng Sa,Thành phố Đà Nẵng,NaN,NaN,NaN
6379,536,51,Huyện,Huyện Lý Sơn,Tỉnh Quảng Ngãi,NaN,NaN,NaN
8196,755,77,Huyện,Huyện Côn Đảo,Tỉnh Bà Rịa - Vũng Tàu,NaN,NaN,NaN


5 cái đảo không có ward nên file ward không có.

# Sáp nhập province

In [15]:
df_sapnhap_province = pd.read_csv('data/sapnhap.bando.com.vn_province.csv')

In [16]:
df_sapnhap_province['isMergedProvince'] = np.where(df_sapnhap_province['truocsapnhap']=='không sáp nhập', False, True)

In [17]:
df_sapnhap_province['truocsapnhap'] = df_sapnhap_province['truocsapnhap'].str.replace(' và ', ', ')
df_sapnhap_province['truocsapnhap'] = df_sapnhap_province['truocsapnhap'].str.split(',\s*')
df_sapnhap_province = df_sapnhap_province.explode('truocsapnhap').reset_index(drop=True)

In [18]:
df_sapnhap_province['truocsapnhap'] = np.where(df_sapnhap_province['isMergedProvince']==True, df_sapnhap_province['truocsapnhap'], df_sapnhap_province['tentinh'])

In [19]:
df_sapnhap_province['truocsapnhap'] = df_sapnhap_province['truocsapnhap'].apply(cap_first)
df_sapnhap_province['tentinh'] = df_sapnhap_province['tentinh'].apply(cap_first)

In [20]:
df_sapnhap_province[~df_sapnhap_province['truocsapnhap'].isin(df_danhmuc['province'])]

,id,mahc,tentinh,dientichkm2,dansonguoi,trungtamhc,kinhdo,vido,truocsapnhap,con,isMergedProvince
0,1,1,Thủ đô Hà Nội,"3.359,80",8.718.000,giữ nguyên,105.698,21.0001,Thủ đô Hà Nội,126 ĐVHC (51 phường và 75 xã),False
14,11,12,Tỉnh Phú Thọ,"9.361,38",4.022.638,Phú Thọ (cũ),105.333,20.7065,Tỉnh Hòa Bình,"148 ĐVHC (15 phường, 133 xã)",True
48,29,29,Thành phố Hồ Chí Minh,"6.772,59",14.002.598,Tp. HCM (cũ),106.638,10.9926,TPHCM,"168 ĐVHC (01 đặc khu, 113 phường, 54 xã)",True


In [21]:
df_sapnhap_province.loc[df_sapnhap_province['truocsapnhap']=='Thủ đô Hà Nội', 'truocsapnhap'] = 'Thành phố Hà Nội'
df_sapnhap_province.loc[df_sapnhap_province['truocsapnhap']=='TPHCM', 'truocsapnhap'] = 'Thành phố Hồ Chí Minh'
df_sapnhap_province.loc[df_sapnhap_province['truocsapnhap']=='Tỉnh Hòa Bình', 'truocsapnhap'] = 'Tỉnh Hoà Bình'

In [22]:
rename_cols = {
    'truocsapnhap': 'province',
    'mahc': 'newProvinceCode',
    'tentinh': 'newProvince',
    'isMergedProvince': 'isMergedProvince',
}
df_sapnhap_province = df_sapnhap_province.rename(columns=rename_cols)[rename_cols.values()]

In [23]:
df_sapnhap_province

,province,newProvinceCode,newProvince,isMergedProvince
0,Thành phố Hà Nội,1,Thủ đô Hà Nội,False
1,Tỉnh Cao Bằng,7,Tỉnh Cao Bằng,False
2,Tỉnh Hà Giang,8,Tỉnh Tuyên Quang,True
3,Tỉnh Tuyên Quang,8,Tỉnh Tuyên Quang,True
4,Tỉnh Yên Bái,9,Tỉnh Lào Cai,True
...,...,...,...,...
58,Thành phố Cần Thơ,33,Thành phố Cần Thơ,True
59,Tỉnh Sóc Trăng,33,Thành phố Cần Thơ,True
60,Tỉnh Hậu Giang,33,Thành phố Cần Thơ,True
61,Tỉnh Bạc Liêu,34,Tỉnh Cà Mau,True


In [24]:
df_danhmuc = pd.merge(df_danhmuc, df_sapnhap_province, on='province', how='left')

In [25]:
df_danhmuc[df_danhmuc['newProvince'].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,isMergedProvince


In [26]:
if df_danhmuc['newProvince'].isna().sum():
    raise Exception('Còn sót province')

# Sáp nhập ward

## Unpivot

In [27]:
df_sapnhap_ward = pd.read_csv('data/sapnhap.bando.com.vn_ward.csv')

In [28]:
df_sapnhap_ward['maxa'].drop_duplicates()

0       1290
1        994
2       1179
3        967
4       1441
        ... 
3316     164
3317     168
3318     160
3319     161
3320     151
Name: maxa, Length: 3312, dtype: int64

In [29]:
# Preview text trong dấu ngoặc
df_sapnhap_ward[df_sapnhap_ward['truocsapnhap'].str.contains('\(')]['truocsapnhap']

0       Xã Đông La, Phường Dương Nội (phần còn lại sau...
1       Phường Quán Thánh, Phường Trúc Bạch, Phường Cử...
2       Xã Ba Vì, Xã Khánh Thượng, Xã Minh Quang (phần...
3       Phường Bạch Mai, Phường Bách Khoa, Phường Quỳn...
5       Xã Kim Đức, Phường Cự Khối (phần còn lại sau k...
                              ...                        
3288    Thị trấn U Minh, Xã Nguyễn Phích, Xã Khánh Thu...
3299    Thị trấn Sông Đốc (bao gồm cụm đảo Hòn Chuối),...
3305    Phường 5 (thành phố Cà Mau), Phường Tân Thành,...
3311    Thị trấn Trần Văn Thời, Xã Khánh Lộc, Xã Phong...
3320         Phường 5 (thành phố Bạc Liêu), Xã Vĩnh Trạch
Name: truocsapnhap, Length: 612, dtype: object

In [30]:
df_sapnhap_ward[df_sapnhap_ward['truocsapnhap'].str.contains('Khánh Bình Tây')]['truocsapnhap']

3264    Xã Khánh Bình Tây (bao gồm Hòn Đá Bạc), Xã Khá...
Name: truocsapnhap, dtype: object

In [31]:
# Remove và replace text
removed_texts = [
    ', Trường bắn TB1',
    ', Khu vực bãi bồi ven biển \(do huyện Kim Sơn quản lý\)',
    '\(phần còn lại.*?\)',
    '\(bao gồm.*?\)',
    '\(một phần.*?\)',
    '\(do huyện.*?\)',
]


replaced_texts = [
    (' và ', ', '),
    ('(thuộc huyện ', '(huyện '),
    ('(huyện Châu Thành, tỉnh Bến Tre)', '(Huyện Châu Thành - Tỉnh Bến Tre)'), # Tạm giữ lại để tính isDividedWard co đúng, Chú ý khi bỏ tỉnh rồi thì merge sẽ bị duplicate Thị trấn Châu Thành, Huyện Châu Thành
    ('(huyện Châu Thành, tỉnh Trà Vinh)', '(Huyện Châu Thành - Tỉnh Trà Vinh)') # Tạm giữ lại để tính isDividedWard co đúng, Chú ý khi bỏ tỉnh rồi thì merge sẽ bị duplicate Thị trấn Châu Thành, Huyện Châu
]

for text in removed_texts:
    df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].str.replace(text, '', regex=True).str.strip()

for old, new in replaced_texts:
    df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].str.replace(old, new, regex=False).str.strip()

In [32]:
# Thêm isMergedWard
df_sapnhap_ward['isMergedWard'] = np.where(df_sapnhap_ward['truocsapnhap']=='Không sáp nhập', False, True)

In [33]:
# Unpivot
df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].apply(fill_district)
df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].str.split(',\s*')
df_sapnhap_ward = df_sapnhap_ward.explode('truocsapnhap').reset_index(drop=True)

In [34]:
df_sapnhap_ward

,id,matinh,ma,tentinh,loai,tenhc,cay,dientichkm2,dansonguoi,trungtamhc,kinhdo,vido,truocsapnhap,maxa,khoa,isMergedWard
0,2,1,2,Thủ đô Hà Nội,xã,An Khánh,1.0002,28.69,102136,"Thôn Lũng Vân, xã An Khánh",105.708,20.98760,Xã Đông La,1290,1An Khánh,True
1,2,1,2,Thủ đô Hà Nội,xã,An Khánh,1.0002,28.69,102136,"Thôn Lũng Vân, xã An Khánh",105.708,20.98760,Phường Dương Nội,1290,1An Khánh,True
2,2,1,2,Thủ đô Hà Nội,xã,An Khánh,1.0002,28.69,102136,"Thôn Lũng Vân, xã An Khánh",105.708,20.98760,Xã An Khánh,1290,1An Khánh,True
3,2,1,2,Thủ đô Hà Nội,xã,An Khánh,1.0002,28.69,102136,"Thôn Lũng Vân, xã An Khánh",105.708,20.98760,Xã La Phù,1290,1An Khánh,True
4,2,1,2,Thủ đô Hà Nội,xã,An Khánh,1.0002,28.69,102136,"Thôn Lũng Vân, xã An Khánh",105.708,20.98760,Xã Song Phương,1290,1An Khánh,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10562,3353,34,3353,tỉnh Cà Mau,xã,Vĩnh Phước,3291.3353,75.50,20014,đang cập nhật,105.410,9.39891,Xã Vĩnh Phú Tây,160,34Vĩnh Phước,True
10563,3354,34,3354,tỉnh Cà Mau,xã,Vĩnh Thanh,3291.3354,37.37,19291,đang cập nhật,105.519,9.35997,Xã Hưng Phú,161,34Vĩnh Thanh,True
10564,3354,34,3354,tỉnh Cà Mau,xã,Vĩnh Thanh,3291.3354,37.37,19291,đang cập nhật,105.519,9.35997,Xã Vĩnh Thanh,161,34Vĩnh Thanh,True
10565,3355,34,3355,tỉnh Cà Mau,phường,Vĩnh Trạch,3291.3355,10.22,23085,đang cập nhật,105.774,9.29729,Phường 5 (thành phố Bạc Liêu),151,34Vĩnh Trạch,True


In [35]:
# Thêm isDividedWard vào df_sapnhap_ward
count_truocsapnhap = df_sapnhap_ward.groupby(['tentinh', 'truocsapnhap'])['truocsapnhap'].count().reset_index(name='count_truocsapnhap')
count_truocsapnhap['isDividedWard'] = np.where(count_truocsapnhap['count_truocsapnhap'] > 1, True, False)
df_sapnhap_ward = pd.merge(df_sapnhap_ward, count_truocsapnhap, on=['tentinh', 'truocsapnhap'], how='left')

In [36]:
# Viết hoa chữ đầu
df_sapnhap_ward['tentinh'] = df_sapnhap_ward['tentinh'].apply(cap_first)

In [37]:
# Lúc merge mới biết nên quay lại đây để sửa
df_sapnhap_ward['tentinh'].replace('Tỉnh Ninh BÌnh', 'Tỉnh Ninh Bình', inplace=True)

In [38]:
# Tách truocsapnhap thành: ward, district, province
df_sapnhap_ward['province'] = df_sapnhap_ward['truocsapnhap'].apply(extract_province)
df_sapnhap_ward['district'] = df_sapnhap_ward['truocsapnhap'].apply(extract_district)
df_sapnhap_ward['ward'] = df_sapnhap_ward['truocsapnhap'].apply(extract_ward)
# Fill ward cũ cho những ward không sáp nhập
df_sapnhap_ward['ward'] = np.where(df_sapnhap_ward['isMergedWard']==False, df_sapnhap_ward['loai'].str.capitalize() + ' ' + df_sapnhap_ward['tenhc'], df_sapnhap_ward['ward'])

In [39]:
df_sapnhap_ward[df_sapnhap_ward['ward'].str.contains('^Huyện')]

,id,matinh,ma,tentinh,loai,tenhc,cay,dientichkm2,dansonguoi,trungtamhc,...,vido,truocsapnhap,maxa,khoa,isMergedWard,count_truocsapnhap,isDividedWard,province,district,ward
1349,1050,4,1050,Thành phố Hải Phòng,đặc khu,Bạch Long Vĩ,1034.1050,3.07,686,Trụ sở huyện Bạch Long Vĩ hiện trạng,...,20.13460,Huyện Bạch Long Vĩ,3309,4Bạch Long Vĩ,True,1,False,NaN,NaN,Huyện Bạch Long Vĩ
5960,1764,19,1764,Tỉnh Quảng Trị,đặc khu,Cồn Cỏ,1752.1764,2.30,139,UBND huyện đảo Cồn Cỏ (cũ),...,17.15925,Huyện Cồn Cỏ,3310,19Cồn Cỏ,True,1,False,NaN,NaN,Huyện Cồn Cỏ
6415,1904,21,1904,Thành phố Đà Nẵng,đặc khu,Hoàng Sa,1872.1904,305.00,9741,"Đường Hoàng Sa, phường Sơn Trà",...,16.56800,Huyện Hoàng Sa,3312,21Hoàng Sa,True,1,False,NaN,NaN,Huyện Hoàng Sa
6734,2015,22,2015,Tỉnh Quảng Ngãi,đặc khu,Lý Sơn,1967.2015,10.39,22000,Trung tâm hành chính huyện Lý Sơn (cũ),...,15.38500,Huyện Lý Sơn,3314,22Lý Sơn,True,1,False,NaN,NaN,Huyện Lý Sơn
8807,2741,29,2741,Thành phố Hồ Chí Minh,đặc khu,Côn Đảo,2687.2741,75.79,6502,"28 đường Tôn Đức Thắng, huyện Côn Đảo, tỉnh Bà...",...,8.69810,Huyện Côn Đảo,3311,29Côn Đảo,True,1,False,NaN,NaN,Huyện Côn Đảo
9931,3125,32,3125,Tỉnh An Giang,đặc khu,Kiên Hải,3084.3125,21.81,17644,UBND huyện Kiên Hải (xã Hòn Tre hiện nay),...,9.80000,Huyện Kiên Hải,3315,32Kiên Hải,True,1,False,NaN,NaN,Huyện Kiên Hải


In [40]:
# Đem các huyện nằm ở ward qua district
df_sapnhap_ward['district'] = np.where(df_sapnhap_ward['ward'].str.contains('^Huyện'), df_sapnhap_ward['ward'], df_sapnhap_ward['district'])
df_sapnhap_ward['ward'] = np.where(df_sapnhap_ward['ward'].str.contains('^Huyện'), np.nan, df_sapnhap_ward['ward'])

In [41]:
# Kiểm tra 2 province có Huyện Châu Thành
df_sapnhap_ward[~df_sapnhap_ward['province'].isna()]

,id,matinh,ma,tentinh,loai,tenhc,cay,dientichkm2,dansonguoi,trungtamhc,...,vido,truocsapnhap,maxa,khoa,isMergedWard,count_truocsapnhap,isDividedWard,province,district,ward
9222,2880,30,2880,Tỉnh Vĩnh Long,xã,Châu Thành,2856.2880,47.71,39104,đang cập nhật,...,9.83937,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,2645,30Châu Thành,True,1,False,Tỉnh Trà Vinh,Huyện Châu Thành,Thị trấn Châu Thành
9374,2934,30,2934,Tỉnh Vĩnh Long,xã,Phú Túc,2856.2934,63.14,62073,UBND huyện Châu Thành,...,10.30100,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,2587,30Phú Túc,True,1,False,Tỉnh Bến Tre,Huyện Châu Thành,Thị trấn Châu Thành


In [42]:
# Rename cho bước tiếp theo
rename_cols = {
    'province': 'province',
    'district': 'district',
    'ward': 'ward',
    'isMergedWard': 'isMergedWard',
    'isDividedWard': 'isDividedWard',

    'tenhc': 'newWard',
    'tentinh': 'newProvince',
    'ma': 'newWardCode',
    'loai': 'newWardType',
    'kinhdo': 'newWardLon',
    'vido': 'newWardLat',
    'dientichkm2': 'newWardAreaKm2',
    'truocsapnhap': 'truocsapnhap',

}

df_sapnhap_ward = df_sapnhap_ward.rename(columns=rename_cols)[rename_cols.values()]

In [43]:
df_sapnhap_ward

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
0,NaN,NaN,Xã Đông La,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã Đông La
1,NaN,NaN,Phường Dương Nội,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Phường Dương Nội
2,NaN,NaN,Xã An Khánh,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã An Khánh
3,NaN,NaN,Xã La Phù,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã La Phù
4,NaN,NaN,Xã Song Phương,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã Song Phương
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10562,NaN,NaN,Xã Vĩnh Phú Tây,True,False,Vĩnh Phước,Tỉnh Cà Mau,3353,xã,105.410,9.39891,75.50,Xã Vĩnh Phú Tây
10563,NaN,NaN,Xã Hưng Phú,True,False,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,9.35997,37.37,Xã Hưng Phú
10564,NaN,NaN,Xã Vĩnh Thanh,True,False,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,9.35997,37.37,Xã Vĩnh Thanh
10565,NaN,thành phố Bạc Liêu,Phường 5,True,False,Vĩnh Trạch,Tỉnh Cà Mau,3355,phường,105.774,9.29729,10.22,Phường 5 (thành phố Bạc Liêu)


## Kiểm tra chính tả 2 bên

Vòng lặp kiểm tra như sau:
- Bước 1: Kiểm tra tìm unit không khớp của bảng df_sapnhap_ward
- Bước 2: Sửa chính tả ở bảng df_sapnhap_ward hoặc df_danhmuc.
- Quay lại Bước 1

In [44]:
def correct_spelling_basic(text):
    spelling_basic = [
        ('Hoà', 'Hòa'),
        ('Hoè', 'Hòe'),
        ('Hoá', 'Hóa'),
        ('Hoả', 'Hỏa'),
        ('Khoá', 'Khóa'),
        ('Thuỵ' ,'Thụy'),
        ('Thuỷ', 'Thủy'),
        ('\' ', "'")
    ]

    if isinstance(text, str):
        for wrong, right in spelling_basic:
            pattern = r'\b' + wrong + r'\b'
            text = re.sub(pattern, right, text)
    return text

In [45]:
for col in ['province', 'district', 'ward']:
    df_danhmuc[col] = df_danhmuc[col].apply(unicode_normalize)
    df_sapnhap_ward[col] = df_sapnhap_ward[col].apply(unicode_normalize)
    df_danhmuc[col] = df_danhmuc[col].apply(correct_spelling_basic)
    df_sapnhap_ward[col] = df_sapnhap_ward[col].apply(correct_spelling_basic)


spelling_danhmuc = [
    # newProvince, ward, correct ward
    ('Tỉnh Hưng Yên', 'Xã An Quí', 'Xã An Quý'),
    ('Tỉnh Hưng Yên', 'Xã Kim Chung', 'Xã Kim Trung'),
    ('Tỉnh Hưng Yên', 'Xã Vũ Quí', 'Xã Vũ Quý'),
    ('Tỉnh Tuyên Quang', 'Xã Quí Quân', 'Xã Quý Quân'),
    ('Tỉnh Tuyên Quang', 'Xã Sán Xả Hồ', 'Xã Sán Sả Hồ'),
    ('Tỉnh Tuyên Quang', 'Xã Xà Phìn', 'Xã Sà Phìn'),
    ('Tỉnh Tuyên Quang', 'Xã Sủng Tráng', 'Xã Sủng Cháng'),
    ('Tỉnh Lào Cai', 'Xã Hát Lìu', 'Xã Hát Lừu'),
    ('Tỉnh Lào Cai', 'Thị trấn NT Liên Sơn', 'Thị trấn Nông trường Liên Sơn'),
    ('Tỉnh Lào Cai', 'Xã Nậm Rạng', 'Xã Nậm Dạng'),
    ('Tỉnh Thái Nguyên', 'Xã Sĩ Bình', 'Xã Sỹ Bình'),
    ('Tỉnh Lạng Sơn', 'Thị trấn NT Thái Bình', 'Thị trấn Nông trường Thái Bình'),
    ('Tỉnh Phú Thọ', 'Xã Săm Khóe', 'Xã Xăm Khòe'),
    ('Tỉnh Điện Biên', 'Xã Xá Tổng', 'Xã Sá Tổng'),
    ('Tỉnh Điện Biên', 'Xã Huổi Lếnh', 'Xã Huổi Lếch'),
    ('Tỉnh Điện Biên', 'Xã Sáng Nhè', 'Xã Xá Nhè'),
    ('Tỉnh Điện Biên', 'Xã Xín Chải', 'Xã Sín Chải'),
    ('Tỉnh Lai Châu', 'Xã Tông Qua Lìn', 'Xã Tung Qua Lìn'),
    ('Tỉnh Lai Châu', 'Xã Lả Nhì Thàng', 'Xã Lản Nhì Thàng'),
    ('Tỉnh Lai Châu', 'Xã Hua Bun', 'Xã Hua Bum'),
    ('Tỉnh Lai Châu', 'Xã Ma Ly Pho', 'Xã Ma Li Pho'),
    ('Tỉnh Sơn La', 'Xã Pú Pẩu', 'Xã Pú Bẩu'),
    ('Tỉnh Sơn La', 'Xã Noong Lay', 'Xã Nong Lay'),
    ('Tỉnh Nghệ An', 'Phường Thu Thuỷ', 'Phường Thu Thủy'),
    ('Tỉnh Nghệ An', 'Xã Hưng Yên', 'Xã Hưng Yên Nam'),
    ('Tỉnh Quảng Trị', 'Xã Đa Krông', 'Xã Đakrông'),
    ('Tỉnh Quảng Trị', 'Thị trấn NT Lệ Ninh', 'Thị trấn Nông trường Lệ Ninh'),
    ('Tỉnh Quảng Trị', 'Thị trấn NT Việt Trung', 'Thị trấn Nông trường Việt Trung'),
    ('Thành phố Đà Nẵng', 'Xã Bha Lê', 'Xã Bha Lêê'),
    ('Thành phố Đà Nẵng', 'Xã Tà Bhinh', 'Xã Tà Bhing'),
    ('Thành phố Đà Nẵng', 'Xã Mà Cooi', 'Xã Mà Cooih'),
    ('Thành phố Đà Nẵng', 'Thị trấn P Rao', 'Thị Trấn Prao'),
    ('Thành phố Đà Nẵng', 'Xã Laêê', 'Xã La Êê'),
    ('Thành phố Đà Nẵng', 'Xã Zuôich', 'Xã Zuôih'),
    ('Tỉnh Quảng Ngãi', 'Xã Bờ Y', 'Xã Pờ Y'),
    ('Tỉnh Quảng Ngãi', 'Xã Măng Buk', 'Xã Măng Bút'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngọc Yêu', 'Xã Ngọk Yêu'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngọc Lây', 'Xã Ngọk Lây'),
    ('Tỉnh Gia Lai', 'Xã Đăk Roong', 'Xã Đăk Rong'),
    ('Tỉnh Gia Lai', 'Xã H Bông', 'Xã HBông'),
    ('Tỉnh Gia Lai', 'Xã Ia Ma Rơn', 'Xã Ia Mrơn'),
    ('Tỉnh Gia Lai', "Xã K'Dang", 'Xã KDang'),
    ('Tỉnh Gia Lai', "Xã H'Neng", 'Xã Hneng'),
    ('Tỉnh Đắk Lắk', "Xã Ea H'đinh", "Xã Ea H'đing"),
    ('Tỉnh Đắk Lắk', 'Xã Cư K Róa', 'Xã Cư Króa'),
    ('Tỉnh Đắk Lắk', 'Xã Cư Jang', 'Xã Cư Yang'),
    ('Tỉnh Đắk Lắk', 'Xã EaBia', 'Xã Ea Bia'),
    ('Tỉnh Đắk Lắk', 'Xã Ealy', 'Xã Ea Ly'),
    ('Tỉnh Đắk Lắk', 'Xã Cư Ê Wi', 'Xã Cư Êwi'),
    ('Tỉnh Đắk Lắk', "Xã Ea R'Bin", 'Xã Ea Rbin'),
    ('Tỉnh Đắk Lắk', 'Xã EaTrol', 'Xã Ea Trol'),
    ('Tỉnh Đắk Lắk', 'Xã Eachà Rang', 'Xã Ea Chà Rang'),
    ('Tỉnh Lào Cai', 'Thị trấn N.T Phong Hải', 'Thị trấn Nông trường Phong Hải'),
    ('Tỉnh Lào Cai', 'Xã Dế Su Phình', 'Xã Dế Xu Phình'),
    ('Tỉnh Lào Cai', 'Xã Nàn Xín', 'Xã Nàn Sín'),
    ('Tỉnh Lào Cai', 'Thị trấn NT Trần Phú', 'Thị trấn Nông trường Trần Phú'),
    ('Tỉnh Lâm Đồng', 'Xã Đạm Bri', "Xã Đamb'ri"),
    ('Tỉnh Lâm Đồng', "Xã Đắk N'Dung", "Xã Đắk N'Drung"),
    ('Tỉnh Lâm Đồng', "Xã Đắk DRông", "Xã Đắk D'rông"),
    ('Tỉnh Lâm Đồng', 'Xã Thuận Quí', 'Xã Thuận Quý'),
    ('Thành phố Hồ Chí Minh', 'Xã Quy Đức', 'Xã Qui Đức'),
    ('Tỉnh Vĩnh Long', 'Xã An Quy', 'Xã An Qui'),
    ('Tỉnh Vĩnh Long', 'Xã Lục Sỹ Thành', 'Xã Lục Sĩ Thành'),
    ('Tỉnh An Giang', 'Xã Ô Long Vỹ', 'Xã Ô Long Vĩ'),
    ('Thành phố Cần Thơ', 'Xã Hòa Tú II', 'Xã Hòa Tú 2'),
    ('Tỉnh Đắk Lắk', 'Xã EaBar', 'Xã Ea Bar'),
    ('Tỉnh Đắk Lắk', "Xã Ea D'rơng", 'Xã Ea Drơng'),
    ('Thủ đô Hà Nội', 'Phường Phương Liên - Trung Tự', 'Phường Phương Liên Trung Tự'),
    ('Thủ đô Hà Nội', 'Phường Văn Miếu - Quốc Tử Giám', 'Phường Văn Miếu Quốc Tử Giám'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Rơ Ông', 'Xã Đăk Rơ Ông'),
    ('Tỉnh Cao Bằng', 'Xã Phong Nậm', 'Xã Phong Nặm'),
    ('Tỉnh Tuyên Quang', 'Xã Nậm Tỵ', 'Xã Nậm Ty'),
    ('Tỉnh Tuyên Quang', 'Thị trấn Phó Bảng', 'Thị trấn Phố Bảng'),
    ('Tỉnh Tuyên Quang', 'Xã Khau Tinh', 'Xã Khâu Tinh'),
    ('Tỉnh Lào Cai', 'Thị trấn Mù Căng Chải', 'Thị trấn Mù Cang Chải'),
    ('Tỉnh Lào Cai', 'Xã Nậm Chầy', 'Xã Nậm Chày'),
    ('Tỉnh Lào Cai', 'Xã Tả Ngải Chồ', 'Xã Tả Ngài Chồ'),
    ('Tỉnh Thái Nguyên', 'Phường Quán Triều', 'Phường Quan Triều'),
    ('Tỉnh Phú Thọ', 'Xã Mi Hòa', 'Xã Mỵ Hòa'),
    ('Tỉnh Điện Biên', 'Xã Luân Giới', 'Xã Luân Giói'),
    ('Tỉnh Điện Biên', 'Xã Pú Nhi', 'Xã Pu Nhi'),
    ('Tỉnh Điện Biên', 'Xã Noọng Hẹt', 'Xã Noong Hẹt'),
    ('Tỉnh Lai Châu', 'Xã Pa Vệ Sử', 'Xã Pa Vệ Sủ'),
    ('Tỉnh Lai Châu', 'Xã Tà Gia', 'Xã Ta Gia'),
    ('Tỉnh Sơn La', 'Xã Nậm Ét', 'Xã Nặm Ét'),
    ('Tỉnh Sơn La' , 'Xã Nà Ơt', 'Xã Nà Ớt'),
    ('Tỉnh Thanh Hóa', 'Xã Vân Âm', 'Xã Vân Am'),
    ('Tỉnh Thanh Hóa', 'Xã Văn Sơn', 'Xã Vân Sơn'),
    ('Tỉnh Nghệ An', 'Xã Lưỡng Minh', 'Xã Lượng Minh'),
    ('Tỉnh Nghệ An', 'Xã Căm Muộn', 'Xã Cắm Muộn'),
    ('Tỉnh Hà Tĩnh', 'Xã Cỗ Đạm', 'Xã Cổ Đạm'),
    ('Thành phố Đà Nẵng', 'Xã Ka Dăng', 'Xã Kà Dăng'),
    ('Thành phố Đà Nẵng', 'Xã Za Hung', 'Xã Zà Hung'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngok Tem', 'Xã Ngọc Tem'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngok Wang', 'Xã Ngọk Wang'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngok Réo', 'Xã Ngọk Réo'),
    ('Tỉnh Quảng Ngãi', 'Xã Rơ Kơi', 'Xã Rờ Kơi'),
    ('Tỉnh Gia Lai', 'Xã Chư Đăng Ya', 'Xã Chư Đang Ya'),
    ('Tỉnh Gia Lai', 'Xã Chư Krêy', 'Xã Chư Krey'),
    ('Tỉnh Gia Lai', 'Thị trấn Đăk Đoa', 'Thị trấn Đak Đoa'),
    ('Tỉnh Gia Lai', 'Xã Đăk Sơmei', 'Xã Đak Sơmei'),
    ('Tỉnh Gia Lai', 'Xã Ia KRai', 'Xã Ia Krăi'),
    ('Tỉnh Gia Lai', 'Xã Đăk Krong', 'Xã Đak Krong'),
    ('Tỉnh Gia Lai', 'Xã Đăk Djrăng', 'Xã Đak Djrăng'),
    ('Tỉnh Gia Lai', 'Xã Ia Mláh', 'Xã Ia Mlăh'),
    ('Tỉnh Đắk Lắk', 'Xã Cuor Đăng', 'Xã Cuôr Đăng'),
    ('Tỉnh Đắk Lắk', 'Xã ĐLiê Ya', 'Xã Dliê Ya'),
    ('Tỉnh Đắk Lắk', 'Xã Ea KNuec', 'Xã Ea Knuếc'),
    ('Tỉnh Đắk Lắk', "Xã Ea M'DRóh", "Xã Ea M'Droh"),
    ('Tỉnh Đắk Lắk', "Xã Chư KBô", "Xã Chứ Kbô"),
    ('Tỉnh Đắk Lắk', "Thị trấn M'Đrắk", "Thị trấn M'Drắk"),
    ('Tỉnh Đắk Lắk', "Xã Ea Puk", "Xã Ea Púk"),
    ('Tỉnh Lâm Đồng', "Xã Đạ M'Rong", "Xã Đạ M'Rông"),
    ('Tỉnh Lâm Đồng', "Xã Buôn Choah", "Xã Buôn Choáh"),
    ('Tỉnh Lâm Đồng', "Xã Đăk DRông", "Xã Đăk D'rông"),
    ('Tỉnh Đồng Nai', "Xã Dak Lua", "Xã Đắc Lua"),
    ('Tỉnh Vĩnh Long', "Xã Qưới Sơn", "Xã Quới Sơn"),
    ('Tỉnh Vĩnh Long', "Xã Vĩnh Kim", "Xã Vinh Kim"),
    ('Tỉnh An Giang', "Phường Vĩnh Ngươn", "Phường Vĩnh Nguơn"),
    ('Thành phố Cần Thơ', "Thị trấn Thanh An", "Thị trấn Thạnh An"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Xú", "Xã Đăk Xú"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Cấm", "Xã Đăk Cấm"),
    ('Tỉnh Quảng Ngãi', "Thị trấn Đắk Hà", "Thị trấn Đăk Hà"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk La", "Xã Đăk La"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Tơ Lung", "Xã Đăk Tơ Lung"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Kôi", "Xã Đăk Kôi"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Hring", "Xã Đăk Hring"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Mar", "Xã Đăk Mar"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Kroong", "Xã Đăk Kroong"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Môn", "Xã Đăk Môn"),
    ('Tỉnh Quảng Ngãi', "Thị trấn Đắk Glei", "Thị trấn Đăk Glei"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Pék", "Xã Đăk Pék"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Nhoong", "Xã Đăk Nhoong"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Man", "Xã Đăk Man"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Blô", "Xã Đăk Blô"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Pxi", "Xã Đăk Pxi"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Blà", "Xã Đăk Blà"),



    ('Tỉnh Quảng Ngãi', 'Xã Đắk Rơ Wa', 'Xã Đăk Rơ Wa'),
    ('Tỉnh Quảng Ngãi', 'Thị trấn Đắk Rve', 'Thị trấn Đăk Rve'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Pne', 'Xã Đăk Pne'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Na', 'Xã Đăk Na'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Sao', 'Xã Đăk Sao'),
    ('Tỉnh Quảng Ngãi', 'Thị trấn Đắk Tô', 'Thị trấn Đăk Tô'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Tờ Kan', 'Xã Đăk Tờ Kan'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ui', 'Xã Đăk Ui'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ang', 'Xã Đăk Ang'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Dục', 'Xã Đăk Dục'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Nông', 'Xã Đăk Nông'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Tờ Re', 'Xã Đăk Tờ Re'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ruồng', 'Xã Đăk Ruồng'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Trăm', 'Xã Đăk Trăm'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Nên', 'Xã Đăk Nên'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ring', 'Xã Đăk Ring'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Tăng', 'Xã Đăk Tăng'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Rơ Nga', 'Xã Đăk Rơ Nga'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Kan', 'Xã Đăk Kan'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Hà', 'Xã Đăk Hà'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Choong', 'Xã Đăk Choong'),
    ('Tỉnh Gia Lai', 'Xã Đắk Kơ Ning', 'Xã Đăk Kơ Ning'),
    ('Tỉnh Lâm Đồng', "Xã Đăk R'Moan", "Xã Đắk R'Moan")



]

# Có vẽ các lỗi sai thường là: qui - quy, s - x, \bch - \btr, nt - nong truong, 2 - II, 4 - IV (Số la mã), ngok - ngoc. Sau này sẽ tạo key cho tính năng parse_address

spelling_sapnhap = [
    # newProvince, ward, correct ward
    ('Tỉnh Ninh Bình', 'Phường Tiên Sơn', 'Xã Tiên Sơn'),
    ('Tỉnh Thái Nguyên', 'Xã Đắc Sơn', 'Phường Đắc Sơn'),
    ('Tỉnh Lai Châu', 'Xã Can Hồ', 'Xã Kan Hồ'),
    ('Tỉnh Sơn La', 'Phường Thảo Ngu Yên', 'Phường Thảo Nguyên'),
    ('Tỉnh Sơn La', 'Xã Xí M Vàng', 'Xã Xím Vàng'),
    ('Tỉnh Nghệ An', 'Xã Thu Thủy', 'Phường Thu Thủy'),
    ('Thành phố Huế', 'Phường Đúc', 'Phường Phường Đúc'),
    ('Thành phố Đà Nẵng', 'Xã Bhalêê', 'Xã Bha Lêê'),
    ('Thành phố Đà Nẵng', 'Xã Avương', 'Xã A Vương'),
    ('Thành phố Đà Nẵng', 'Xã Gari', 'Xã Ga Ri'),
    ('Thành phố Đà Nẵng', 'Xã Axan', 'Xã A Xan'),
    ('Thành phố Đà Nẵng', 'Xã Atiêng', 'Xã A Tiêng'),
    ('Thành phố Đà Nẵng', 'Xã Anông', 'Xã A Nông'),
    ('Tỉnh Quảng Ngãi', 'Lê Lợi', 'Phường Lê Lợi'),
    ('Tỉnh Quảng Ngãi', 'Nguyễn Trãi', 'Phường Nguyễn Trãi'),
    ('Tỉnh Quảng Ngãi', 'Xã Đăk Plô', 'Xã Đăk Blô'),
    ('Tỉnh Quảng Ngãi', 'Phổ Hòa', 'Phường Phổ Hòa'),
    ('Tỉnh Quảng Ngãi', 'Phổ Minh', 'Phường Phổ Minh'),
    ('Tỉnh Quảng Ngãi', 'Phổ Vinh', 'Phường Phổ Vinh'),
    ('Tỉnh Quảng Ngãi', 'Phổ Ninh', 'Phường Phổ Ninh'),
    ('Tỉnh Gia Lai', 'Xã Chơ GLong', 'Xã Chơ Long'),
    ('Tỉnh Gia Lai', 'Xã Adơk', 'Xã A Dơk'),
    ('Tỉnh Gia Lai', 'Xã Ia Dreh', 'Ia HDreh'),
    ('Tỉnh Đắk Lắk', 'Krông Na', 'Xã Krông Na'),
    ('Tỉnh Đắk Lắk', 'Xã Cư Dliê Mnông', "Xã Cư Dliê M'nông"),
    ('Tỉnh Lâm Đồng', 'Xã ĐamBri', "Xã Đamb'ri"),
    ('Tỉnh Lâm Đồng', "Xã Cư K'nia", 'Xã Cư Knia'),
    ('Tỉnh Lâm Đồng', 'Thị trấn Mađaguôi', 'Thị trấn Ma Đa Guôi'),
    ('Tỉnh Lâm Đồng', 'Xã Mađaguôi', 'Xã Ma Đa Guôi'),
    ('Tỉnh Lào Cai', 'Xã Tà Xi Láng', 'Xã Tà Si Láng'),
    ('Tỉnh Gia Lai', 'Ia HDreh', 'Xã Ia HDreh'),
    ('Tỉnh Lâm Đồng', "Xã Đưng K'Nớ", 'Xã Đưng KNớ'),
    ('Tỉnh Lâm Đồng', "Xã N'Thôn Hạ", "Xã N'Thol Hạ"),
    ('Tỉnh Tây Ninh', 'Thủ Thừa', 'Thị trấn Thủ Thừa'),
    ('Tỉnh Tây Ninh', 'Phường IV', 'Phường 4'),
    ('Tỉnh Đồng Nai', 'Xã Xuân An', 'Phường Xuân An'),
    ('Tỉnh Đồng Nai', 'Xã Xuân Bình', 'Phường Xuân Bình'),
    ('Thành phố Hồ Chí Minh', 'Rạch Ông', 'Phường Rạch Ông'),
    ('Tỉnh Đồng Tháp', 'Xã Mỹ Quí', 'Xã Mỹ Quý'),
    ('Thành phố Cần Thơ', 'Xã Tân Lộc', 'Phường Tân Lộc'),
    ('Tỉnh Lào Cai', 'Xã Thẳm Dương', 'Xã Thẩm Dương'),
    ('Tỉnh Sơn La', 'Xã Nặm Păm', 'Xã Nậm Păm'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngọk Tem', 'Xã Ngọc Tem'),
    ('Tỉnh Gia Lai', 'Xã Ia Drang', 'Xã Ia Drăng'),
    ('Tỉnh Gia Lai', 'Xã Đak Rong', 'Xã Đăk Rong'),
    ('Tỉnh Gia Lai', 'Xã Ia Krái', 'Xã Ia Krăi'),
    ('Tỉnh Gia Lai', 'Xã Ia Trôk', 'Xã Ia Trok'),
    ('Tỉnh Gia Lai', 'Xã Đak Trôi', 'Xã Đăk Trôi'),
    ('Tỉnh Gia Lai', 'Xã Ia Mlah', 'Xã Ia Mlăh'),
    ('Tỉnh Đắk Lắk', 'Xã Ea Khăl', 'Xã Ea Khal'),
    ('Tỉnh Đồng Nai', "Xã Đak Lua", "Xã Đắc Lua"),
    ('Tỉnh Đồng Nai', "Xã Đăk Ơ", "Xã Đak Ơ"),




]


for s in spelling_danhmuc:
    newProvince, ward, WardCorrect = s
    df_danhmuc.loc[(df_danhmuc['newProvince']==newProvince) & (df_danhmuc['ward'].fillna('').str.lower()==ward.lower()), 'ward'] = WardCorrect

for s in spelling_sapnhap:
    newProvince, ward, WardCorrect = s
    df_sapnhap_ward.loc[(df_sapnhap_ward['newProvince']==newProvince) & (df_sapnhap_ward['ward'].fillna('').str.lower()==ward.lower()), 'ward'] = WardCorrect

In [46]:
# Case đặc biệt, phải JOIN vào rồi mới thấy thiếu
df_sapnhap_ward.loc[(df_sapnhap_ward['newProvince']=='Tỉnh Đồng Nai') & (df_sapnhap_ward['newWard']=='Long Khánh') & (df_sapnhap_ward['ward']=='Xã Xuân Hòa'), 'ward'] = 'Phường Xuân Hòa'
df_sapnhap_ward.loc[(df_sapnhap_ward['newProvince']=='Tỉnh Đồng Nai') & (df_sapnhap_ward['newWard']=='Long Khánh') & (df_sapnhap_ward['ward']=='Xã Phú Bình'), 'ward'] = 'Phường Phú Bình'

In [47]:
for col in ['district', 'ward']:
    df_danhmuc[f'{col}Ud'] = df_danhmuc[col].apply(unidecode_pro)
    df_sapnhap_ward[f'{col}Ud'] = df_sapnhap_ward[col].apply(unidecode_pro)

    df_danhmuc[f'{col}Lower'] = df_danhmuc[col].apply(lower_safely)
    df_sapnhap_ward[f'{col}Lower'] = df_sapnhap_ward[col].apply(lower_safely)

In [48]:
key1 = 'keyNewProvinceProvinceDistrictWard' # Đặc biệt, chỉ có 2 province ^_^
key2 = 'keyNewProvinceDistrictWard' # Trường hợp này sẽ sai nhiều vì dùng fill_district chưa chắc đúng. Cứ kiểm tra chính tả cho key3 key4 thôi là được. Sau đó khi merge bằng key2 phần nào không match chính là key4b
key3 = 'keyNewProvinceDistrict'
key4 = 'keyNewProvinceWard' # key4a


# # Mục đích để check chính tả sẽ khác mục đích dể merge
# for df in [df_sapnhap_ward, df_danhmuc]:
#     df[key1] = np.where((~df['province'].isna()) & (~df['district'].isna()) & (~df['ward'].isna()), df['newProvince'] + '_' + df['province'] + '_' + df['districtUd'] + '_' + df['wardUd'], np.nan)
#
# for df in [df_sapnhap_ward, df_danhmuc]:
#     df[key2] = np.where((~df['district'].isna()) & (~df['ward'].isna()), df['newProvince'] + '_' + df['districtUd'] + '_' + df['wardUd'], np.nan)
#
# for df in [df_sapnhap_ward, df_danhmuc]:
#     df[key3] = np.where((~df['district'].isna()), df['newProvince'] + '_' + df['districtUd'], np.nan) # Khi merge cần thêm điều kiện ward là nan
#
# for df in [df_sapnhap_ward, df_danhmuc]:
#     df[key4] = np.where((~df['ward'].isna()), df['newProvince'] + '_' + df['wardUd'], np.nan) # Khi merge cần thêm điều kiện district là nan


for df in [df_sapnhap_ward, df_danhmuc]:
    df[key1] = np.where((~df['province'].isna()) & (~df['district'].isna()) & (~df['ward'].isna()), df['newProvince'] + '_' + df['province'] + '_' + df['districtLower'] + '_' + df['wardLower'], np.nan)

df_danhmuc[key2] = np.where((~df_danhmuc['district'].isna()) & (~df_danhmuc['ward'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['districtLower'] + '_' + df_danhmuc['wardLower'], np.nan)
df_sapnhap_ward[key2] = np.where((~df_sapnhap_ward['district'].isna()) & (~df_sapnhap_ward['ward'].isna()) & (df_sapnhap_ward['province'].isna()), df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtLower'] + '_' + df_sapnhap_ward['wardLower'], np.nan) # Khi merge thêm province là nan

df_danhmuc[key3] = np.where((~df_danhmuc['district'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['districtLower'], np.nan)
df_sapnhap_ward[key3] = np.where((~df_sapnhap_ward['district'].isna()) & (df_sapnhap_ward['ward'].isna()), df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtLower'], np.nan) # Khi merge cần thêm điều kiện ward là nan

df_danhmuc[key4] = np.where((~df_danhmuc['ward'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['wardLower'], np.nan)
df_sapnhap_ward[key4] = np.where((~df_sapnhap_ward['ward'].isna()) & (df_sapnhap_ward['district'].isna()), df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['wardLower'], np.nan) # Khi merge cần thêm điều kiện district là nan

In [49]:
# Preview (2 province duy nhất trong dataset này)
df_sapnhap_ward[~df_sapnhap_ward[key1].isna()]

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
9222,Tỉnh Trà Vinh,Huyện Châu Thành,Thị trấn Châu Thành,True,False,Châu Thành,Tỉnh Vĩnh Long,2880,xã,106.337,...,47.71,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Trà Vinh_huyện châu thành_...,NaN,NaN,NaN
9374,Tỉnh Bến Tre,Huyện Châu Thành,Thị trấn Châu Thành,True,False,Phú Túc,Tỉnh Vĩnh Long,2934,xã,106.318,...,63.14,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Bến Tre_huyện châu thành_t...,NaN,NaN,NaN


In [50]:
# Preview (Các đảo thì không có ward)
df_danhmuc[df_danhmuc[key1].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,isMergedProvince,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
3582,318,31,Huyện,Huyện Bạch Long Vĩ,Thành phố Hải Phòng,NaN,NaN,NaN,4,Thành phố Hải Phòng,True,huyen bach long vi,huyện bạch long vĩ,NaN,NaN,NaN,NaN,Thành phố Hải Phòng_huyện bạch long vĩ,NaN
5794,471,45,Huyện,Huyện Cồn Cỏ,Tỉnh Quảng Trị,NaN,NaN,NaN,19,Tỉnh Quảng Trị,True,huyen con co,huyện cồn cỏ,NaN,NaN,NaN,NaN,Tỉnh Quảng Trị_huyện cồn cỏ,NaN
5975,498,48,Huyện,Huyện Hoàng Sa,Thành phố Đà Nẵng,NaN,NaN,NaN,21,Thành phố Đà Nẵng,True,huyen hoang sa,huyện hoàng sa,NaN,NaN,NaN,NaN,Thành phố Đà Nẵng_huyện hoàng sa,NaN
6379,536,51,Huyện,Huyện Lý Sơn,Tỉnh Quảng Ngãi,NaN,NaN,NaN,22,Tỉnh Quảng Ngãi,True,huyen ly son,huyện lý sơn,NaN,NaN,NaN,NaN,Tỉnh Quảng Ngãi_huyện lý sơn,NaN
8196,755,77,Huyện,Huyện Côn Đảo,Tỉnh Bà Rịa - Vũng Tàu,NaN,NaN,NaN,29,Thành phố Hồ Chí Minh,True,huyen con dao,huyện côn đảo,NaN,NaN,NaN,NaN,Thành phố Hồ Chí Minh_huyện côn đảo,NaN


In [51]:
# Check Key 1
df_sapnhap_ward[~df_sapnhap_ward[key1].isin(df_danhmuc[key1])]
# Key 1 is valid fully

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard


In [52]:
# Check Key 3
df_sapnhap_ward[~df_sapnhap_ward[key3].isin(df_danhmuc[key3]) & (~df_sapnhap_ward[key3].isna())][['newProvince', 'district', 'districtUd']].drop_duplicates()
# Key 3 is valid fully

,newProvince,district,districtUd


In [53]:
# Check key 4
df_sapnhap_ward[~df_sapnhap_ward[key4].isin(df_danhmuc[key4]) & (~df_sapnhap_ward[key4].isna())][['newProvince', 'district', 'ward', 'wardUd', 'isMergedWard']].drop_duplicates()
# Chỉ còn "Xã Tân Phước" là không tồn tại ở Tỉnh Tuyên Quang và Tỉnh Hà Giang. Có lẽ dữ liệu của sapnhap bị sai chổ này

,newProvince,district,ward,wardUd,isMergedWard
2738,Tỉnh Tuyên Quang,NaN,Xã Tân Phước,xa tan phuoc,True


In [54]:
condition1 = (df_danhmuc['newProvince'] == 'Tỉnh Tuyên Quang')
condition2 = (df_danhmuc['wardUd'].fillna('').str.contains('\b.a. phu'))
df_danhmuc[condition1 & condition2][['newProvince', 'province', 'district', 'ward', 'wardUd']].drop_duplicates()

,newProvince,province,district,ward,wardUd


## Kiểm tra trước khi merge

In [55]:
for df in [df_sapnhap_ward, df_danhmuc]:
    df[key1] = np.where((~df['province'].isna()) & (~df['district'].isna()) & (~df['ward'].isna()), df['newProvince'] + '_' + df['province'] + '_' + df['districtUd'] + '_' + df['wardUd'], np.nan)

df_danhmuc[key2] = np.where((~df_danhmuc['district'].isna()) & (~df_danhmuc['ward'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['districtUd'] + '_' + df_danhmuc['wardUd'], np.nan)
df_sapnhap_ward[key2] = np.where((~df_sapnhap_ward['district'].isna()) & (~df_sapnhap_ward['ward'].isna()) & (df_sapnhap_ward['province'].isna()), df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtUd'] + '_' + df_sapnhap_ward['wardUd'], np.nan) # Khi merge thêm province là nan

df_danhmuc[key3] = np.where((~df_danhmuc['district'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['districtUd'], np.nan)
df_sapnhap_ward[key3] = np.where((~df_sapnhap_ward['district'].isna()) & (df_sapnhap_ward['ward'].isna()), df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtUd'], np.nan) # Khi merge cần thêm điều kiện ward là nan

df_danhmuc[key4] = np.where((~df_danhmuc['ward'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['wardUd'], np.nan)
df_sapnhap_ward[key4] = np.where((~df_sapnhap_ward['ward'].isna()) & (df_sapnhap_ward['district'].isna()), df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['wardUd'], np.nan) # Khi merge cần thêm điều kiện district là nan

In [56]:
sapnhap1 = df_sapnhap_ward[~df_sapnhap_ward[key1].isna()]
sapnhap1

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
9222,Tỉnh Trà Vinh,Huyện Châu Thành,Thị trấn Châu Thành,True,False,Châu Thành,Tỉnh Vĩnh Long,2880,xã,106.337,...,47.71,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Trà Vinh_huyen chau thanh_...,NaN,NaN,NaN
9374,Tỉnh Bến Tre,Huyện Châu Thành,Thị trấn Châu Thành,True,False,Phú Túc,Tỉnh Vĩnh Long,2934,xã,106.318,...,63.14,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Bến Tre_huyen chau thanh_t...,NaN,NaN,NaN


In [57]:
sapnhap3 = df_sapnhap_ward[~df_sapnhap_ward[key3].isna()]
sapnhap3

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
1349,NaN,Huyện Bạch Long Vĩ,NaN,True,False,Bạch Long Vĩ,Thành phố Hải Phòng,1050,đặc khu,107.7307,...,3.07,Huyện Bạch Long Vĩ,huyen bach long vi,huyện bạch long vĩ,NaN,NaN,NaN,NaN,Thành phố Hải Phòng_huyen bach long vi,NaN
5960,NaN,Huyện Cồn Cỏ,NaN,True,False,Cồn Cỏ,Tỉnh Quảng Trị,1764,đặc khu,107.3390,...,2.30,Huyện Cồn Cỏ,huyen con co,huyện cồn cỏ,NaN,NaN,NaN,NaN,Tỉnh Quảng Trị_huyen con co,NaN
6415,NaN,Huyện Hoàng Sa,NaN,True,False,Hoàng Sa,Thành phố Đà Nẵng,1904,đặc khu,112.1350,...,305.00,Huyện Hoàng Sa,huyen hoang sa,huyện hoàng sa,NaN,NaN,NaN,NaN,Thành phố Đà Nẵng_huyen hoang sa,NaN
6734,NaN,Huyện Lý Sơn,NaN,True,False,Lý Sơn,Tỉnh Quảng Ngãi,2015,đặc khu,109.1150,...,10.39,Huyện Lý Sơn,huyen ly son,huyện lý sơn,NaN,NaN,NaN,NaN,Tỉnh Quảng Ngãi_huyen ly son,NaN
8807,NaN,Huyện Côn Đảo,NaN,True,False,Côn Đảo,Thành phố Hồ Chí Minh,2741,đặc khu,106.6115,...,75.79,Huyện Côn Đảo,huyen con dao,huyện côn đảo,NaN,NaN,NaN,NaN,Thành phố Hồ Chí Minh_huyen con dao,NaN
9931,NaN,Huyện Kiên Hải,NaN,True,False,Kiên Hải,Tỉnh An Giang,3125,đặc khu,104.6320,...,21.81,Huyện Kiên Hải,huyen kien hai,huyện kiên hải,NaN,NaN,NaN,NaN,Tỉnh An Giang_huyen kien hai,NaN


In [58]:
sapnhap4a = df_sapnhap_ward[~df_sapnhap_ward[key4].isna()]
sapnhap4a

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
0,NaN,NaN,Xã Đông La,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,...,28.69,Xã Đông La,NaN,NaN,xa dong la,xã đông la,NaN,NaN,NaN,Thủ đô Hà Nội_xa dong la
1,NaN,NaN,Phường Dương Nội,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,...,28.69,Phường Dương Nội,NaN,NaN,phuong duong noi,phường dương nội,NaN,NaN,NaN,Thủ đô Hà Nội_phuong duong noi
2,NaN,NaN,Xã An Khánh,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,...,28.69,Xã An Khánh,NaN,NaN,xa an khanh,xã an khánh,NaN,NaN,NaN,Thủ đô Hà Nội_xa an khanh
3,NaN,NaN,Xã La Phù,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,...,28.69,Xã La Phù,NaN,NaN,xa la phu,xã la phù,NaN,NaN,NaN,Thủ đô Hà Nội_xa la phu
4,NaN,NaN,Xã Song Phương,True,False,An Khánh,Thủ đô Hà Nội,2,xã,105.708,...,28.69,Xã Song Phương,NaN,NaN,xa song phuong,xã song phương,NaN,NaN,NaN,Thủ đô Hà Nội_xa song phuong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10561,NaN,NaN,Xã Phước Long,True,False,Vĩnh Phước,Tỉnh Cà Mau,3353,xã,105.410,...,75.50,Xã Phước Long,NaN,NaN,xa phuoc long,xã phước long,NaN,NaN,NaN,Tỉnh Cà Mau_xa phuoc long
10562,NaN,NaN,Xã Vĩnh Phú Tây,True,False,Vĩnh Phước,Tỉnh Cà Mau,3353,xã,105.410,...,75.50,Xã Vĩnh Phú Tây,NaN,NaN,xa vinh phu tay,xã vĩnh phú tây,NaN,NaN,NaN,Tỉnh Cà Mau_xa vinh phu tay
10563,NaN,NaN,Xã Hưng Phú,True,False,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,...,37.37,Xã Hưng Phú,NaN,NaN,xa hung phu,xã hưng phú,NaN,NaN,NaN,Tỉnh Cà Mau_xa hung phu
10564,NaN,NaN,Xã Vĩnh Thanh,True,False,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,...,37.37,Xã Vĩnh Thanh,NaN,NaN,xa vinh thanh,xã vĩnh thanh,NaN,NaN,NaN,Tỉnh Cà Mau_xa vinh thanh


In [59]:
sapnhap2 = df_sapnhap_ward[(~df_sapnhap_ward[key2].isna()) & (df_sapnhap_ward[key2].isin(df_danhmuc[key2]))]
sapnhap2

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
20,NaN,quận Hai Bà Trưng,Phường Bạch Mai,True,False,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,...,2.95,Phường Bạch Mai (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong bach mai,phường bạch mai,NaN,Thủ đô Hà Nội_quan hai ba trung_phuong bach mai,NaN,NaN
21,NaN,quận Hai Bà Trưng,Phường Bách Khoa,True,False,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,...,2.95,Phường Bách Khoa (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong bach khoa,phường bách khoa,NaN,Thủ đô Hà Nội_quan hai ba trung_phuong bach khoa,NaN,NaN
22,NaN,quận Hai Bà Trưng,Phường Quỳnh Mai,True,False,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,...,2.95,Phường Quỳnh Mai (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong quynh mai,phường quỳnh mai,NaN,Thủ đô Hà Nội_quan hai ba trung_phuong quynh mai,NaN,NaN
23,NaN,quận Hai Bà Trưng,Phường Minh Khai,True,True,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,...,2.95,Phường Minh Khai (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong minh khai,phường minh khai,NaN,Thủ đô Hà Nội_quan hai ba trung_phuong minh khai,NaN,NaN
75,NaN,huyện Phú Xuyên,Xã Tân Dân,True,False,Chuyên Mỹ,Thủ đô Hà Nội,13,xã,105.884,...,35.54,Xã Tân Dân (huyện Phú Xuyên),huyen phu xuyen,huyện phú xuyên,xa tan dan,xã tân dân,NaN,Thủ đô Hà Nội_huyen phu xuyen_xa tan dan,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10521,NaN,thành phố Cà Mau,Phường 5,True,False,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,...,48.42,Phường 5 (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,phuong 5,phường 5,NaN,Tỉnh Cà Mau_thanh pho ca mau_phuong 5,NaN,NaN
10522,NaN,thành phố Cà Mau,Phường Tân Thành,True,False,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,...,48.42,Phường Tân Thành (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,phuong tan thanh,phường tân thành,NaN,Tỉnh Cà Mau_thanh pho ca mau_phuong tan thanh,NaN,NaN
10523,NaN,thành phố Cà Mau,Xã Tân Thành,True,False,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,...,48.42,Xã Tân Thành (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,xa tan thanh,xã tân thành,NaN,Tỉnh Cà Mau_thanh pho ca mau_xa tan thanh,NaN,NaN
10524,NaN,thành phố Cà Mau,Phường 7,True,True,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,...,48.42,Phường 7 (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,phuong 7,phường 7,NaN,Tỉnh Cà Mau_thanh pho ca mau_phuong 7,NaN,NaN


In [60]:
sapnhap4b = df_sapnhap_ward[(~df_sapnhap_ward[key2].isna()) & (~df_sapnhap_ward[key2].isin(df_danhmuc[key2]))]
sapnhap4b['district'] = np.nan
sapnhap4b['districtUd'] = np.nan
sapnhap4b[key4] = np.where((~sapnhap4b['ward'].isna()) & (sapnhap4b['district'].isna()), sapnhap4b['newProvince'] + '_' + sapnhap4b['wardUd'], np.nan)
sapnhap4b

,province,district,ward,isMergedWard,isDividedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,...,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
181,NaN,NaN,Phường Thạch Bàn,True,False,Gia Lâm,Thủ đô Hà Nội,29,xã,105.949,...,25.72,Phường Thạch Bàn (huyện Gia Lâm),NaN,huyện gia lâm,phuong thach ban,phường thạch bàn,NaN,Thủ đô Hà Nội_huyen gia lam_phuong thach ban,NaN,Thủ đô Hà Nội_phuong thach ban
242,NaN,NaN,Phường Tây Tựu,True,True,Hoài Đức,Thủ đô Hà Nội,38,xã,105.705,...,16.73,Phường Tây Tựu (huyện Hoài Đức),NaN,huyện hoài đức,phuong tay tuu,phường tây tựu,NaN,Thủ đô Hà Nội_huyen hoai duc_phuong tay tuu,NaN,Thủ đô Hà Nội_phuong tay tuu
243,NaN,NaN,Xã Tân Lập,True,False,Hoài Đức,Thủ đô Hà Nội,38,xã,105.705,...,16.73,Xã Tân Lập (huyện Hoài Đức),NaN,huyện hoài đức,xa tan lap,xã tân lập,NaN,Thủ đô Hà Nội_huyen hoai duc_xa tan lap,NaN,Thủ đô Hà Nội_xa tan lap
259,NaN,NaN,Phường Hoàng Liệt,True,False,Hoàng Liệt,Thủ đô Hà Nội,40,phường,105.832,...,4.04,Phường Hoàng Liệt (huyện Thanh Trì),NaN,huyện thanh trì,phuong hoang liet,phường hoàng liệt,NaN,Thủ đô Hà Nội_huyen thanh tri_phuong hoang liet,NaN,Thủ đô Hà Nội_phuong hoang liet
357,NaN,NaN,Xã Văn Khê,True,False,Mê Linh,Thủ đô Hà Nội,56,xã,105.733,...,34.97,Xã Văn Khê (huyện Đan Phượng),NaN,huyện đan phượng,xa van khe,xã văn khê,NaN,Thủ đô Hà Nội_huyen dan phuong_xa van khe,NaN,Thủ đô Hà Nội_xa van khe
358,NaN,NaN,Xã Mê Linh,True,False,Mê Linh,Thủ đô Hà Nội,56,xã,105.733,...,34.97,Xã Mê Linh (huyện Đan Phượng),NaN,huyện đan phượng,xa me linh,xã mê linh,NaN,Thủ đô Hà Nội_huyen dan phuong_xa me linh,NaN,Thủ đô Hà Nội_xa me linh
359,NaN,NaN,Xã Đại Thịnh,True,False,Mê Linh,Thủ đô Hà Nội,56,xã,105.733,...,34.97,Xã Đại Thịnh (huyện Đan Phượng),NaN,huyện đan phượng,xa dai thinh,xã đại thịnh,NaN,Thủ đô Hà Nội_huyen dan phuong_xa dai thinh,NaN,Thủ đô Hà Nội_xa dai thinh
584,NaN,NaN,Phường Tây Tựu,True,True,Tây Tựu,Thủ đô Hà Nội,92,phường,105.739,...,7.54,Phường Tây Tựu (huyện Hoài Đức),NaN,huyện hoài đức,phuong tay tuu,phường tây tựu,NaN,Thủ đô Hà Nội_huyen hoai duc_phuong tay tuu,NaN,Thủ đô Hà Nội_phuong tay tuu
620,NaN,NaN,Xã Đại Mạch,True,False,Thiên Lộc,Thủ đô Hà Nội,98,xã,105.771,...,27.96,Xã Đại Mạch (huyện Mê Linh),NaN,huyện mê linh,xa dai mach,xã đại mạch,NaN,Thủ đô Hà Nội_huyen me linh_xa dai mach,NaN,Thủ đô Hà Nội_xa dai mach
621,NaN,NaN,Xã Kim Nỗ,True,False,Thiên Lộc,Thủ đô Hà Nội,98,xã,105.771,...,27.96,Xã Kim Nỗ (huyện Mê Linh),NaN,huyện mê linh,xa kim no,xã kim nỗ,NaN,Thủ đô Hà Nội_huyen me linh_xa kim no,NaN,Thủ đô Hà Nội_xa kim no


In [61]:
if not (sapnhap1.shape[0] + sapnhap2.shape[0] + sapnhap3.shape[0] + sapnhap4a.shape[0] + sapnhap4b.shape[0]) == df_sapnhap_ward.shape[0]:
    raise Exception('Chia tệp chưa MECE')

## Tiến hành merge

In [62]:
df_convert1 = pd.merge(df_danhmuc, sapnhap1, on=['newProvince', 'province', 'districtUd', 'wardUd'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert2 = pd.merge(df_danhmuc, sapnhap2, on=['newProvince', 'districtUd', 'wardUd'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert3 = pd.merge(df_danhmuc, sapnhap3, on=['newProvince', 'districtUd'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert4a = pd.merge(df_danhmuc, sapnhap4a, on=['newProvince', 'wardUd'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert4b = pd.merge(df_danhmuc, sapnhap4b, on=['newProvince', 'wardUd'], how='inner', suffixes=('', '_DUPLICATED'))

In [63]:
print(df_convert1.shape[0], sapnhap1.shape[0])
print(df_convert2.shape[0], sapnhap2.shape[0])
print(df_convert3.shape[0], sapnhap3.shape[0])
print(df_convert4a.shape[0], sapnhap4a.shape[0])
print(df_convert4b.shape[0], sapnhap4b.shape[0])


2 2
904 903
9 6
9856 9604
53 52


In [64]:
df_convert = pd.concat([df_convert1, df_convert2, df_convert3, df_convert4a, df_convert4b], ignore_index=True).drop_duplicates()
df_convert.drop(columns=[col for col in df_convert.columns if '_DUPLICATED' in col or 'key' in col], inplace=True)
df_convert.drop_duplicates(inplace=True)

In [66]:
df_convert[df_convert['wardUd'].fillna('').str.contains('phuong 15') & df_convert['districtUd'].fillna('').str.contains('tan binh')]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardLower,isMergedWard,isDividedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
602,766,79,Quận,Quận Tân Bình,Thành phố Hồ Chí Minh,27007.0,Phường,Phường 15,29,Thành phố Hồ Chí Minh,...,phường 15,True,True,Tân Bình,2807,phường,106.642,10.8006,2.12,Phường 15 (quận Tân Bình)
603,766,79,Quận,Quận Tân Bình,Thành phố Hồ Chí Minh,27007.0,Phường,Phường 15,29,Thành phố Hồ Chí Minh,...,phường 15,True,True,Tân Sơn,2819,phường,106.650,10.8224,10.12,Phường 15 (quận Tân Bình)


In [67]:
df_convert_full = pd.merge(df_danhmuc, df_convert, on=['newProvince', 'province', 'districtUd', 'wardUd'], how='left', suffixes=('', '_DUPLICATED')).drop_duplicates()
df_convert_full .drop(columns=[col for col in df_convert_full.columns if '_DUPLICATED' in col or 'key' in col], inplace=True)
df_convert_full.drop_duplicates(inplace=True)

In [69]:
df_convert_full[df_convert_full['newWard'].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardLower,isMergedWard,isDividedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
977,34,2,Huyện,Huyện Bắc Quang,Tỉnh Hà Giang,1174.0,Xã,Xã Thượng Bình,8,Tỉnh Tuyên Quang,...,xã thượng bình,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378,75,8,Huyện,Huyện Yên Sơn,Tỉnh Tuyên Quang,2488.0,Xã,Xã Trung Môn,8,Tỉnh Tuyên Quang,...,xã trung môn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_sapnhap_ward[df_sapnhap_ward['wardUd'].fillna('').str.contains('xuan hoa')]

In [70]:
not_exist_in_sapnhap = [
    'Xã Thượng Bình', # newProvince = "Tỉnh Tuyên Quang" -> newWard = "Xã Đồng Tâm" https://address-converter.io.vn/
    'Xã Trung Môn', # newProvince = "Tỉnh Tuyên Quang" -> newWard = "Phường Minh Xuân" https://address-converter.io.vn/
]
exist_in_sapnhap = [
    'Phường Xuân Hòa', # ward = 'Xã Xuân Hòa' newWard = 'Long Khánh', newProvince = 'Tỉnh Đồng Nai'
    'Phường Phú Bình', # ward = 'Xã Phú Bình' newWard = 'Long Khánh', newProvince = 'Tỉnh Đồng Nai'
]

In [71]:
fill_map = [
    {
        'province': 'Tỉnh Hà Giang',
        'district': 'Huyện Bắc Quang',
        'ward': 'Xã Thượng Bình',
        'newProvince': 'Tỉnh Tuyên Quang',
        'newWard': 'Đồng Tâm'
    },
    {
        'province': 'Tỉnh Tuyên Quang',
        'district': 'Huyện Yên Sơn',
        'ward': 'Xã Trung Môn',
        'newProvince': 'Tỉnh Tuyên Quang',
        'newWard': 'Minh Xuân'
    },
]
fill_cols = ['isMergedWard', 'isDividedWard', 'newWard', 'newWardCode', 'newWardType', 'newWardLon', 'newWardLat', 'newWardAreaKm2']
for record in fill_map:
    miss_condition = (df_convert_full['province'] == record['province']) & (df_convert_full['district'] == record['district']) & (df_convert_full['ward'] == record['ward'])
    fill_condition = (df_convert_full['newProvince'] == record['newProvince']) & (df_convert_full['newWard'] == record['newWard'])

    fill_values = df_convert_full.loc[fill_condition, fill_cols].iloc[0]
    df_convert_full.loc[miss_condition, fill_cols] = fill_values.values

In [80]:
df_convert_full.to_csv('data/danhmuc_and_sapnhap.csv', index=False)

In [ ]:
df_convert_full['newWardCode']